In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [15]:
import pandas as pd
import numpy as np
import datetime
pd.set_option('display.max_colwidth', None) #显示单元格完整信息
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

user = pd.read_csv(r'C:\Users\User\Desktop\SEO_0807\会员列表导出.csv',encoding='gbk')
firChargeUser = pd.read_csv(r'C:\Users\User\Desktop\SEO_0807\会员首存报表.csv',encoding='gbk')
data = pd.read_excel(r'C:\Users\User\Desktop\SEO_0807\data\今日数据.xlsx')
daili = pd.read_excel(r'C:\Users\User\Desktop\SEO_0807\SEO每日模板-每日更新.xlsx','代理总表')

shuju = pd.DataFrame({'人员':['Paddy', 'Tony', 'Max', 'Martin', 'Zed', 'Hugo', 'Aber', 'DK', 'Ben','当日汇总'],
                      '日期':(datetime.datetime.now()+datetime.timedelta(days=-3)).strftime('%Y/%m/%d'),
                      '发送IP':0,
                      '接受IP':0,
                      '对比昨天(总IP)':0,
                      '对比前3天均值(总IP)':0,
                      '对比前5天均值(总IP)':0,
                      '对比前7天均值(总IP)':0,
                      '对比昨天(总注册)':0,
                      '对比前3天均值(总注册)':0,
                      '对比前5天均值(总注册)':0,
                      '对比前7天均值(总注册)':0,
                      '对比昨天(总开户)':0,
                      '对比前3天均值(总开户)':0,
                      '对比前5天均值(总开户)':0,
                      '对比前7天均值(总开户)':0})

shuju.set_index('人员',inplace = True)

data['IP']=pd.to_numeric(data['IP'],errors='coerce').replace(np.nan,0).astype('int64')
grp=data.groupby('网站名(domain)').agg({'IP':sum})

shuju.loc['Paddy','发送IP']=grp.loc['paddy.com','IP']
shuju.loc['Paddy','接受IP']=grp.loc['paddy.bty','IP']
shuju.loc['Tony','发送IP']=grp.loc['tonyb.com','IP']/2
shuju.loc['Tony','接受IP']=grp.loc['tonyb.com','IP']
shuju.loc['Max','发送IP']=grp.loc['mulu.com','IP']
shuju.loc['Max','接受IP']=grp.loc['max.bty','IP']
shuju.loc['Martin','发送IP']=grp.loc['redquan.com','IP']
shuju.loc['Martin','接受IP']=grp.loc['martin.bty','IP']
shuju.loc['Zed','发送IP']=grp.loc['zed.com','IP']
shuju.loc['Zed','接受IP']=grp.loc['zed.bty','IP']
shuju.loc['Hugo','发送IP']=grp.loc['hugo.com','IP']
# shuju.loc['Hugo','接受IP']=grp.loc['hugo.bty','IP']
shuju.loc['Hugo','接受IP']=0

shuju.loc['Aber','发送IP']=grp.loc['aber.com','IP']/2
shuju.loc['Aber','接受IP']=grp.loc['aber.bty','IP']
shuju.loc['DK','发送IP']=grp.loc['dk.com','IP']/2
shuju.loc['DK','接受IP']=grp.loc['dk.bty','IP']
shuju.loc['Ben','发送IP']=grp.loc['ben.com','IP']/2
shuju.loc['Ben','接受IP']=grp.loc['ben.bty','IP']
shuju.loc['当日汇总','发送IP']=shuju['发送IP'].sum()
shuju.loc['当日汇总','接受IP']=shuju['接受IP'].sum()

shuju['日期'] = pd.to_datetime(shuju['日期'])
shuju.insert(1,'人员2',shuju.index)
shuju['人员2']=shuju['人员2'].str.lower()

merge_user = pd.merge(user,daili,how = 'left',left_on='代理',right_on='代理线')
grpSEO = merge_user.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
grpSEO.rename(columns={'seo变化数据团队':'注册'},inplace=True)
grpSEO.reset_index(inplace=True)
grpSEO['人员2'] = grpSEO['seo变化数据团队'].str.lower()
grpSEO.set_index('seo变化数据团队',inplace=True)


# 第1次merge前，重置索引
shuju.reset_index(inplace=True)

shuju = pd.merge(shuju,grpSEO,on='人员2')
shuju['注册率'] = shuju['注册']/shuju['发送IP']*100
shuju.loc['当日汇总','注册']=shuju['注册'].sum()
shuju.loc['当日汇总','注册率']=shuju['注册率'].sum()

merge_charge = pd.merge(firChargeUser,daili,how='left',left_on='所属代理',right_on='代理线')
grpCHARGE = merge_charge.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
grpCHARGE= grpCHARGE.rename(columns={'seo变化数据团队':'开户'})
grpCHARGE.reset_index(inplace=True)
grpCHARGE['seo变化数据团队']=grpCHARGE['seo变化数据团队'].str.lower()
grpCHARGE= grpCHARGE.rename(columns={'seo变化数据团队':'人员2'})
# 第2次merge
shuju = pd.merge(shuju,grpCHARGE,how='left',on='人员2')
shuju['转化率'] = shuju['开户']/shuju['注册']*100
shuju.loc['当日汇总','开户']=shuju['开户'].sum()
shuju.loc['当日汇总','转化率']=shuju['转化率'].sum()

grp3  = merge_charge[merge_charge['注册时间'].str[:9]==merge_charge['交易时间'].str[:9]].groupby('seo变化数据团队').agg({'seo变化数据团队':len})
grp3.rename(columns = {'seo变化数据团队':'当日注册并开户'},inplace=True)
grp3.reset_index(inplace=True)
grp3['seo变化数据团队'] =grp3['seo变化数据团队'].str.lower()
grp3.rename(columns = {'seo变化数据团队':'人员2'},inplace=True)
# 第3次merge
shuju  = pd.merge(shuju,grp3,how='left',on='人员2')
shuju['当日注册激活率'] = shuju['当日注册并开户']/shuju['注册']*100
shuju.loc['当日汇总','当日注册并开户']=shuju['当日注册并开户'].sum()
shuju.loc['当日汇总','当日注册激活率']=shuju['当日注册激活率'].sum()

C:\ProgramData\anaconda3\envs\schedule\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [16]:
shuju

,人员,日期,人员2,发送IP,接受IP,对比昨天(总IP),对比前3天均值(总IP),对比前5天均值(总IP),对比前7天均值(总IP),对比昨天(总注册),对比前3天均值(总注册),对比前5天均值(总注册),对比前7天均值(总注册),对比昨天(总开户),对比前3天均值(总开户),对比前5天均值(总开户),对比前7天均值(总开户),注册,注册率,开户,转化率,当日注册并开户,当日注册激活率
0,Paddy,2023-08-07,paddy,0.0,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,inf,2.0,66.666667,2.0,66.666667
1,Tony,2023-08-07,tony,568.0,1136.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,2.640845,3.0,20.000000,3.0,20.000000
2,Max,2023-08-07,max,273.0,225.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,3.296703,NaN,NaN,NaN,NaN
3,Martin,2023-08-07,martin,3445.0,1072.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81.0,2.351234,24.0,29.629630,18.0,22.222222
4,Zed,2023-08-07,zed,4853.0,329.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.453328,8.0,36.363636,8.0,36.363636
5,Hugo,2023-08-07,hugo,103.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,15.533981,5.0,31.250000,2.0,12.500000
6,Aber,2023-08-07,aber,7325.0,562.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,0.668942,13.0,26.530612,9.0,18.367347
7,DK,2023-08-07,dk,1525.0,2160.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,3.409836,5.0,9.615385,5.0,9.615385
8,Ben,2023-08-07,ben,2565.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.662768,6.0,35.294118,3.0,17.647059
9,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,264.0,inf,NaN,NaN,NaN,NaN


In [6]:
merge_user = pd.merge(user,daili,how = 'left',left_on='代理',right_on='代理线')
grpSEO = merge_user.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
grpSEO.rename(columns={'seo变化数据团队':'注册'},inplace=True)
grpSEO.reset_index(inplace=True)
grpSEO['人员2'] = grpSEO['seo变化数据团队'].str.lower()
grpSEO.set_index('seo变化数据团队',inplace=True)
grpSEO

,注册,人员2
seo变化数据团队,,
178直播,3,178直播
Admin邀请,2,admin邀请
Cyrus,2,cyrus
DK,52,dk
GT-B直播,16,gt-b直播
GT-越南线,2,gt-越南线
GT-骨头,63,gt-骨头
MM,1,mm
SEO掉线用户转线,8,seo掉线用户转线


In [7]:
merge_charge = pd.merge(firChargeUser,daili,how='left',left_on='所属代理',right_on='代理线')
grpCHARGE = merge_charge.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
grpCHARGE= grpCHARGE.rename(columns={'seo变化数据团队':'开户'})
grpCHARGE.reset_index(inplace=True)
grpCHARGE['seo变化数据团队']=grpCHARGE['seo变化数据团队'].str.lower()
grpCHARGE= grpCHARGE.rename(columns={'seo变化数据团队':'人员2'})
grpCHARGE

,人员2,开户
0,admin邀请,1
1,cyrus,1
2,dk,5
3,gt-b直播,5
4,gt-骨头,20
5,seo掉线用户转线,16
6,seo邀请,4
7,soco越南直播,46
8,aber,13
9,ben,6


In [8]:
merge_charge = pd.merge(firChargeUser,daili,how='left',left_on='所属代理',right_on='代理线')
grpCHARGE = merge_charge.groupby('seo变化数据团队').agg({'seo变化数据团队':len})
grpCHARGE= grpCHARGE.rename(columns={'seo变化数据团队':'开户'})
grpCHARGE.reset_index(inplace=True)
grpCHARGE['人员2']=grpCHARGE['seo变化数据团队'].str.lower()
grpCHARGE.set_index('seo变化数据团队',inplace=True)
grpCHARGE

,开户,人员2
seo变化数据团队,,
Admin邀请,1,admin邀请
Cyrus,1,cyrus
DK,5,dk
GT-B直播,5,gt-b直播
GT-骨头,20,gt-骨头
SEO掉线用户转线,16,seo掉线用户转线
SEO邀请,4,seo邀请
SOCO越南直播,46,soco越南直播
aber,13,aber


In [10]:
grp3  = merge_charge[merge_charge['注册时间'].str[:9]==merge_charge['交易时间'].str[:9]].groupby('seo变化数据团队').agg({'seo变化数据团队':len})
grp3.rename(columns = {'seo变化数据团队':'当日注册并开户'},inplace=True)
grp3.reset_index(inplace=True)
grp3['人员2'] =grp3['seo变化数据团队'].str.lower()
grp3.set_index('seo变化数据团队',inplace=True)
grp3

,当日注册并开户,人员2
seo变化数据团队,,
DK,5,dk
GT-B直播,2,gt-b直播
GT-骨头,15,gt-骨头
SEO掉线用户转线,1,seo掉线用户转线
SEO邀请,3,seo邀请
SOCO越南直播,21,soco越南直播
aber,9,aber
ben,3,ben
duobo,1,duobo


In [11]:
shuju

,人员,日期,人员2,发送IP,接受IP,对比昨天(总IP),对比前3天均值(总IP),对比前5天均值(总IP),对比前7天均值(总IP),对比昨天(总注册),对比前3天均值(总注册),对比前5天均值(总注册),对比前7天均值(总注册),对比昨天(总开户),对比前3天均值(总开户),对比前5天均值(总开户),对比前7天均值(总开户),注册,注册率,开户,转化率,当日注册并开户,当日注册激活率
0,Paddy,2023-08-07,paddy,0.0,51.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,inf,2.0,66.666667,2.0,66.666667
1,Tony,2023-08-07,tony,568.0,1136.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,2.640845,3.0,20.000000,3.0,20.000000
2,Max,2023-08-07,max,273.0,225.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,3.296703,NaN,NaN,NaN,NaN
3,Martin,2023-08-07,martin,3445.0,1072.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81.0,2.351234,24.0,29.629630,18.0,22.222222
4,Zed,2023-08-07,zed,4853.0,329.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.453328,8.0,36.363636,8.0,36.363636
5,Hugo,2023-08-07,hugo,103.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,15.533981,5.0,31.250000,2.0,12.500000
6,Aber,2023-08-07,aber,7325.0,562.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,0.668942,13.0,26.530612,9.0,18.367347
7,DK,2023-08-07,dk,1525.0,2160.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,3.409836,5.0,9.615385,5.0,9.615385
8,Ben,2023-08-07,ben,2565.0,298.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.662768,6.0,35.294118,3.0,17.647059
9,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,264.0,inf,NaN,NaN,NaN,NaN


In [12]:
shuju = pd.merge(shuju,grpSEO,on='人员2')
shuju['注册率'] = shuju['注册']/shuju['发送IP']*100
shuju.loc['当日汇总','注册']=shuju['注册'].sum()
shuju.loc['当日汇总','注册率']=shuju['注册率'].sum()
# 第2次merge
shuju = pd.merge(shuju,grpCHARGE,how='left',on='人员2')
shuju['转化率'] = shuju['开户']/shuju['注册']*100
shuju.loc['当日汇总','开户']=shuju['开户'].sum()
shuju.loc['当日汇总','转化率']=shuju['转化率'].sum()
shuju  = pd.merge(shuju,grp3,how='left',on='人员2')
shuju['当日注册激活率'] = shuju['当日注册并开户']/shuju['注册']*100
shuju.loc['当日汇总','当日注册并开户']=shuju['当日注册并开户'].sum()
shuju.loc['当日汇总','当日注册激活率']=shuju['当日注册激活率'].sum()
shuju

KeyError: '注册'